In [1]:
import json
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
# print(os.getcwd())
import dist_utilities as dist_ult
# import noise_est_utilities as ne_ult
importlib.reload(dist_ult)
os.chdir(current_dir)
import pickle
from qiskit.result import Result
from qiskit.visualization import plot_histogram, plot_distribution
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import numpy as np
from copy import deepcopy
from qiskit.result import QuasiDistribution

In [2]:
with open("ideal_counts_hardware.json", "r") as f:
    ideal_counts=json.load(f)
with open("noisy_counts_hardware.json", "r") as f:
    noisy_counts=json.load(f)
with open("noisy_noise_est_dist_hardware.json", "r") as f:
    noisy_noise_est_dist=json.load(f)
with open("ideal_noise_est_counts_hardware.json", "r") as f:
    ideal_noise_est_counts=json.load(f)
print("ideal: ", ideal_noise_est_counts)
print("noise est dist: ", noisy_noise_est_dist)
print("noisy counts: ", noisy_counts)
print("ideal counts: ", ideal_counts)
col_label=list(dict(ideal_noise_est_counts).keys())[0]
# corrected_counts=ne_ult.get_corrected_counts_from_counts(noisy_counts, noisy_noise_est_dist, list(ideal_noise_est_counts.keys())[0])

# print(corrected_counts)

ideal:  {'10101': 200000}
noise est dist:  {'00101': 0.087695, '00001': 0.025565, '10001': 0.041555, '10110': 0.02718, '11101': 0.08874, '10010': 0.009465, '00011': 0.01002, '11000': 0.01254, '10100': 0.07427, '10000': 0.021625, '11100': 0.041625, '10101': 0.15893, '11110': 0.01591, '00111': 0.02984, '10111': 0.052145, '11111': 0.029985, '01100': 0.02488, '00000': 0.01496, '10011': 0.015285, '11011': 0.00902, '01101': 0.04982, '00110': 0.01632, '00100': 0.043085, '11010': 0.00573, '11001': 0.023405, '01110': 0.009865, '01111': 0.018105, '01001': 0.015325, '01011': 0.00635, '00010': 0.00711, '01000': 0.009105, '01010': 0.004545}
noisy counts:  {'11010': 6049, '11111': 17189, '00010': 5148, '10100': 5569, '01001': 5369, '10011': 6169, '11101': 7341, '01010': 5524, '00000': 5102, '01000': 5294, '01111': 6657, '10101': 5742, '10001': 5431, '11011': 7528, '00001': 5042, '00111': 5799, '11000': 5794, '01011': 5839, '10010': 5595, '01110': 5890, '11100': 6362, '11001': 6019, '01100': 5499, '1

In [3]:
# plot_histogram([ideal_counts, noisy_counts, corrected_counts], number_to_keep=5, legend=["ideal counts", "noisy counts", "corrected counts"], figsize=(20,20), filename="plot_sim.jpg")


In [4]:
num_qubits=len(list(noisy_counts.items())[0][0])

# if 2**10<2**num_qubits:
num_keep=2**15#2**10#num_qubits#2**10
if num_keep>2**num_qubits:
    num_keep=2**num_qubits
noisy_counts_orig=deepcopy(noisy_counts)
print("num keep ", num_keep)
noisy_noise_est_dist=dist_ult.convert_to_first_col(noisy_noise_est_dist, col_label)
noisy_noise_est_dist=dist_ult.filter_counts(noisy_noise_est_dist, num_keep)
noisy_counts=dist_ult.filter_counts(noisy_counts, num_keep)
all_keys, noisy_noise_est_dist, noisy_counts=dist_ult.fill_counts_range(noisy_noise_est_dist, noisy_counts, num_keep, num_qubits)

noisy_noise_est_vec=sorted(list(noisy_noise_est_dist.items()), key=lambda x: int(x[0], 2))
noisy_noise_est_vec=[elem[1] for elem in noisy_noise_est_vec]
counts_vec=sorted(list(noisy_counts.items()), key=lambda x: int(x[0], 2))
counts_vec=[elem[1] for elem in counts_vec]
# print(len(noisy_noise_est_vec))
# print(len(counts_vec))
quasi_corrected_counts=dist_ult.get_quasi_corrected(noisy_noise_est_vec, counts_vec)
quasi_corrected_counts=dist_ult.label_quasis(quasi_corrected_counts, all_keys)
quasi_method=QuasiDistribution(quasi_corrected_counts)
quasi_corrected_counts=quasi_method.nearest_probability_distribution().binary_probabilities(num_qubits)

num keep  32
max exist  31
min exist  0
size before  32
32
32


In [5]:
fig=plot_histogram([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=5, 
               legend=["ideal", "raw", "corrected"], title="Grover Search 5 Qubits Quantum Hardware")
fig.savefig("plot_grover5_counts_hardware_ibm_marrakesh.jpg", bbox_inches="tight")


In [6]:
fig=plot_distribution([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=5, 
                  legend=["ideal", "raw", "corrected"], title="Grover Search 5 Qubits Quantum Hardware") #figsize=(15,15))
ax=fig.axes[0]
ax.set_ylabel("Probability")
fig.savefig("plot_grover5_dist_hardware_ibm_marrakesh.jpg", bbox_inches="tight")

In [7]:
from qiskit.quantum_info import hellinger_fidelity
print(hellinger_fidelity(ideal_counts, quasi_corrected_counts))
print(hellinger_fidelity(ideal_counts, noisy_counts_orig))
print(ideal_counts)
print(quasi_corrected_counts)
print(noisy_counts_orig)

0.7488414715665496
0.10194198467363527
{'10110': 2, '11111': 199839, '10001': 11, '01000': 9, '11000': 3, '10000': 6, '11110': 2, '00100': 8, '00000': 3, '00001': 4, '10100': 7, '01100': 9, '01101': 7, '01011': 5, '01010': 6, '10011': 6, '10010': 4, '01110': 5, '01111': 7, '00010': 4, '11010': 1, '11101': 3, '00111': 4, '11100': 3, '00101': 6, '00110': 7, '11011': 5, '00011': 4, '10111': 9, '11001': 4, '01001': 2, '10101': 5}
{'10011': 242.244809913242, '11100': 386.477849583815, '10101': 909.317036273860, '01101': 3831.39076183990, '10110': 11720.1819072147, '01110': 12150.5463345860, '00111': 22604.2714308528, '11111': 148155.569869736}
{'11010': 6049, '11111': 17189, '00010': 5148, '10100': 5569, '01001': 5369, '10011': 6169, '11101': 7341, '01010': 5524, '00000': 5102, '01000': 5294, '01111': 6657, '10101': 5742, '10001': 5431, '11011': 7528, '00001': 5042, '00111': 5799, '11000': 5794, '01011': 5839, '10010': 5595, '01110': 5890, '11100': 6362, '11001': 6019, '01100': 5499, '10111